In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix

from rapidgbm import RapidGBMTuner

import mplhep as hep
hep.style.use([hep.style.ATLAS])

import seaborn as sns

In [2]:
data1 = pd.read_parquet('/groups/hep/kinch/pred_e_m/samples_processed/Zee1.parquetoutput.parquet')
data2 = pd.read_parquet('/groups/hep/kinch/pred_e_m/samples_processed/Zee2.parquetoutput.parquet')
data3 = pd.read_parquet('/groups/hep/kinch/pred_e_m/samples_processed/Zee3.parquetoutput.parquet')

In [3]:
data = pd.concat([data1, data2, data3])

In [4]:
drop_list = ['runNumber', 'eventNumber', 'mcEventNumber', 'el_index', 'el_firstEgMotherTruthType',
       'el_firstEgMotherTruthOrigin', 'truthType',
       'truthOrigin', 'neflowisol20','truthel_e_dressed', 
       'truthel_pt_dressed', 'truthel_eta_dressed',
       'truthel_phi_dressed', 'truthel_m', 'truthel_px', 'truthel_py',
       'truthel_pz', 'truthel_E', 'truthel_pdgId', 'truthel_ptcone30',
       'truthel_etcone20', 'truthel_ParticleOrigin', 'truthel_Classification',
       'truthel_barcode', 'truthel_status', 'truthel_nPhotons_dressed', 'truth_dR', 'm']

In [5]:
# split the data

X_train, X_test, y_train, y_test = train_test_split(data, data['truthel_E'], test_size=0.2, random_state=42)

X_test_full = X_test.copy()
X_train.drop(drop_list, axis=1, inplace=True)
X_test.drop(drop_list, axis=1, inplace=True)
print(X_train.columns)


Index(['pt', 'eta', 'phi', 'charge', 'f1', 'DNN_pel', 'DNN_pcf', 'DNN_ppc',
       'DNN_phf', 'DNN_ple', 'DNN_plh', 'el_DFCommonElectronsECIDS',
       'el_DFCommonElectronsECIDSResult', 'GSF_dR', 'GSF_d0', 'GSF_z0',
       'GSF_theta', 'GSF_phi', 'GSF_qOverP', 'GSF_chiSquared', 'GSF_var_0',
       'GSF_var_1', 'GSF_var_2', 'GSF_var_3', 'GSF_var_4', 'InDet_dR',
       'InDet_d0', 'InDet_z0', 'InDet_theta', 'InDet_phi', 'InDet_var_0',
       'InDet_var_1', 'InDet_var_2', 'InDet_var_3', 'InDet_var_4',
       'ptvarcone20', 'topoetcone20', 'topoetcone40',
       'el_ptvarcone30_Nonprompt_All_MaxWeightTTVALooseCone_pt1000',
       'el_ptcone20_Nonprompt_All_MaxWeightTTVALooseCone_pt1000',
       'el_ptvarcone30_Nonprompt_All_MaxWeightTTVALooseCone_pt1000_CloseByCorr',
       'el_topoetcone20_CloseByCorr',
       'el_ptcone20_Nonprompt_All_MaxWeightTTVALooseCone_pt1000_CloseByCorr',
       'el_core57cellsEnergyCorrection', 'el_topoetcone20ptCorrection',
       'el_ptcone20_Nonprompt_All_Max

In [6]:
tuner = RapidGBMTuner(metric='mae', trials=30, refit=True, verbosity=1, visualization=True, seed=414243)#, grid='h,learning_rate')

tuner.grid['max_depth'] = (0, 100)
tuner.grid['learning_rate'] = (0.00001, 0.1)
tuner.grid['drop_rate'] = (0.05,0.4)
# Fit tuner for each set of input_data
tuner.fit(X_train, y_train)


 * Initiating LGBMTuner.fit
     . Settings:
     .. Trying 30 trials
     .. Evaluation metric: mae 
     .. Study direction: minimize mae



In [10]:
plt.rcdefaults()
hep.style.use([hep.style.ATLAS])

# Plot the learning curve
predictions = tuner.predict(X_test)

TypeError: RapidGBMTuner.predict() got an unexpected keyword argument 'return_std'